In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def maxAction(Q, state):
    values = np.array([Q[state,a] for a in range(4)])
    action = np.argmax(values)
    return action

def choseandcheck(Q, a, observation, EPSILON, env, s):
    rand = np.random.random()
    if rand < (1-EPSILON):
        a = maxAction(Q,s)
    else:
        a = np.random.randint(0,4)
   # print(env.step(a), a)
    new_state, reward, done, _ = env.step(a)
    observation_ = numpy_transformer_light(new_state)
    #print(observation_, '\n\n', observation)
    while observation==observation_:
        #print("REACHED)")
        Q[s,a] -= 1
        rand = np.random.random()
        if rand < (1-EPSILON):
            a = maxAction(Q,s)
        else:
            a = np.random.randint(0,4)
            
        new_state, reward, done, _ = env.step(a)
        observation_ = numpy_transformer_light(new_state)
        
    return a, observation_, reward, done


def numpy_transformer(matrix):
    lst = []
    for i in range(len(matrix)):
            for j in range(len(matrix[0])):
                    lst.append(int(matrix[i][j]))
    return tuple(lst)

def numpy_transformer_light(matrix):
    lst = []
    for i in range(len(matrix)):
            for j in range(len(matrix[0])):
                    lst.append(int(matrix[i][j]))
    return lst


def get_max(state):
    max_value = max(state)
    return max_value

def plotLearning(x, y, x_label, y_label,title):
    plt.scatter(x,y, label = 'skitscat', color = 'k', s=25 , marker = 'o')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)


In [ ]:
import gym 
import numpy as np
import gym_2048
import random

if __name__ == '__main__':
    env = gym.make('2048-v0')

    #model hyperparameters
    ALPHA = 0.1
    GAMMA = 0.99
    EPSILON = 1.0
    #this variable counts how many times we have won.
    won = 0
    #value we want to reach due to memory restrictions.
    GOAL = 512
    max_value = 0
    num_won = []
    max_values = []
    avg_values = []
    max_total = 0

    #construct state space
 
    states = []
    Q = {}

    numGames = 500
    BATCH_SIZE = int(numGames/100)
    totalRewards = []
    for i in range(numGames):
        game_won = False
        observation = numpy_transformer_light(env.reset())
        s = tuple(observation)
        #check if observation isn't already in states
        if observation not in states:
            states.append(observation)
            for a in range(4):
                #we changed the state to a to avoid a TypeError,
                #because lists aren't hashable.S
                Q[tuple(s),a] = 0
        rand = np.random.random()

        done = False
        epRewards = 0
        while not done:      
            # choice and validity check to avoid dead ends (loops)
            a, observation_, reward, done = choseandcheck(Q, a, observation, EPSILON, env, s)

            s_ = tuple(observation_)
            if observation_ not in states:
                states.append(observation_)
                for a in range(4):
                    #we changed the state to a to avoid a TypeError,
                    #because lists aren't hashable.
                    Q[tuple(s_),a] = 0
                    

            rand = np.random.random()
            if rand < (1-EPSILON):
                a_ = maxAction(Q,s_)
            else: 
                a_ = np.random.randint(0,4)
            epRewards += reward
            Q[s,a] = Q[s,a] + ALPHA*(reward + GAMMA*Q[s_,a_] - Q[s,a])
            
            # no more needed. that does not check for validity of move. we incorporated thi in choseandcheck
            # s,a = s_,a_

            #checks if the GOAL is reached. Sets the done to True to avoid KeyError (a higher state can be reached
            # but we don't want to reach it, because we don't have in in our state_space.)
            if get_max(observation_)==GOAL:
                game_won = True
            
            if get_max(observation_)>max_value:
                max_value = get_max(observation_)
                
            observation=observation_
        
        
        if game_won == True:
            won+= 1

        if EPSILON > 0.0002:
            EPSILON -= 2/(numGames)  
        else:
            EPSILON = 0.0002
        totalRewards.append(epRewards)
        max_total += get_max(observation)
        
        if (i+1)%(BATCH_SIZE) == 0:
            new = won/(BATCH_SIZE)
            num_won.append(new)
            max_values.append(max_value)
            average = max_total/BATCH_SIZE
            avg_values.append(average)
            won = 0
            max_value = 0
            max_total = 0
            print("Episode: " + str(i+1))
            
            

In [ ]:
x = [j for j in range(1,101)]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

plotLearning(x,num_won,'Batch','Won games in percentage', 'Win-Statistic')
# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(x,num_won)

# use line_kws to set line label for legend
ax = sns.regplot(x=x, y=num_won, color='b', 
 line_kws={'label':"y={0:.7f}x+{1:.7f}".format(slope,intercept)})

# plot legend
ax.legend()

plt.show()

plotLearning(x,max_values,'Batch','Max_values', 'Max-Statistic')
# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(x,max_values)

# use line_kws to set line label for legend
ax = sns.regplot(x=x, y=max_values, color='b', 
 line_kws={'label':"y={0:.7f}x+{1:.7f}".format(slope,intercept)})

# plot legend
ax.legend()

plt.show()


plotLearning(x,avg_values,'Batch','Avg_values', 'Average-Max-Statistic')
# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(x,avg_values)

# use line_kws to set line label for legend
ax = sns.regplot(x=x, y=avg_values, color='b', 
 line_kws={'label':"y={0:.7f}x+{1:.7f}".format(slope,intercept)})

# plot legend
ax.legend()

plt.show()


In [ ]:
round = 2
if __name__ == '__main__':
    env = gym.make('2048-v0')

    #model hyperparameters (commetn what doesn need to be rewritten)
    #ALPHA = 0.1
    GAMMA = 0.99
    #As already some knowledge, less randomness. still decreasing. reduce start with each repetition
    EPSILON = 1/round
    #this variable counts how many times we have won.
    won = 0
    #value we want to reach due to memory restrictions.
    #GOAL = 512
    max_value = 0
    #num_won = []
    #max_values = []
    #avg_values = []
    max_total = 0

    #construct state space
 
    #states = []
    #Q = {}

    #numGames = 5000
    BATCH_SIZE = int(numGames/100)
    
    for i in range(numGames):
        game_won = False
        observation = numpy_transformer_light(env.reset())
        s = tuple(observation)
        #check if observation isn't already in states
        if observation not in states:
            states.append(observation)
            for a in range(4):
                #we changed the state to a to avoid a TypeError,
                #because lists aren't hashable.S
                Q[tuple(s),a] = 0
        rand = np.random.random()

        done = False
        epRewards = 0
        while not done:      
            # choice and validity check to avoid dead ends (loops)
            a, observation_, reward, done = choseandcheck(Q, a, observation, EPSILON, env, s)

            s_ = tuple(observation_)
            if observation_ not in states:
                states.append(observation_)
                for a in range(4):
                    #we changed the state to a to avoid a TypeError,
                    #because lists aren't hashable.
                    Q[tuple(s_),a] = 0
                    

            rand = np.random.random()
            if rand < (1-EPSILON):
                a_ = maxAction(Q,s_)
            else: 
                a_ = np.random.randint(0,4)
            epRewards += reward
            Q[s,a] = Q[s,a] + ALPHA*(reward + GAMMA*Q[s_,a_] - Q[s,a])
            
            # no more needed. that does not check for validity of move. we incorporated thi in choseandcheck
            # s,a = s_,a_

            #checks if the GOAL is reached. Sets the done to True to avoid KeyError (a higher state can be reached
            # but we don't want to reach it, because we don't have in in our state_space.)
            if get_max(observation_)==GOAL:
                game_won = True
            
            if get_max(observation_)>max_value:
                max_value = get_max(observation_)
                
            observation=observation_
        
        
        if game_won == True:
            won+= 1

        if EPSILON > 0.0002:
            EPSILON -= 2/(numGames)  
        else:
            EPSILON = 0.0002
        totalRewards.append(epRewards)
        max_total += get_max(observation)
        
        if (i+1)%(BATCH_SIZE) == 0:
            new = won/(BATCH_SIZE)
            num_won.append(new)
            max_values.append(max_value)
            average = max_total/BATCH_SIZE
            avg_values.append(average)
            won = 0
            max_value = 0
            max_total = 0
            print("Episode: " + str(i+1))
            

In [ ]:
x = [j for j in range(1,101)]
lower_bound= (round-1)*100
upper_bound= round*100

In [ ]:
plotLearning(x,num_won[lower_bound:],'Batch','Won games in percentage', 'Win-Statistic')
# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(x,num_won[lower_bound:])

# use line_kws to set line label for legend
ax = sns.regplot(x=x, y=num_won[lower_bound:], color='b', 
 line_kws={'label':"y={0:.7f}x+{1:.7f}".format(slope,intercept)})

# plot legend
ax.legend()

plt.show()

plotLearning(x,max_values[lower_bound:],'Batch','Max_values', 'Max-Statistic')
# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(x,max_values[lower_bound:])

# use line_kws to set line label for legend
ax = sns.regplot(x=x, y=max_values[lower_bound:], color='b', 
 line_kws={'label':"y={0:.7f}x+{1:.7f}".format(slope,intercept)})

# plot legend
ax.legend()

plt.show()


plotLearning(x,avg_values[lower_bound:],'Batch','Avg_values', 'Average-Max-Statistic')
# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(x,avg_values[lower_bound:])

# use line_kws to set line label for legend
ax = sns.regplot(x=x, y=avg_values[lower_bound:], color='b', 
 line_kws={'label':"y={0:.7f}x+{1:.7f}".format(slope,intercept)})

# plot legend
ax.legend()

plt.show()


In [ ]:
round += 1
if __name__ == '__main__':
    env = gym.make('2048-v0')

    #model hyperparameters (commetn what doesn need to be rewritten)
    #ALPHA = 0.1
    GAMMA = 0.99
    #As already some knowledge, less randomness. still decreasing. reduce start with each repetition
    EPSILON = 1/round
    #this variable counts how many times we have won.
    won = 0
    #value we want to reach due to memory restrictions.
    #GOAL = 512
    max_value = 0
    #num_won = []
    #max_values = []
    #avg_values = []
    max_total = 0

    #construct state space
 
    #states = []
    #Q = {}

    #numGames = 5000
    BATCH_SIZE = int(numGames/100)
    
    for i in range(numGames):
        game_won = False
        observation = numpy_transformer_light(env.reset())
        s = tuple(observation)
        #check if observation isn't already in states
        if observation not in states:
            states.append(observation)
            for a in range(4):
                #we changed the state to a to avoid a TypeError,
                #because lists aren't hashable.S
                Q[tuple(s),a] = 0
        rand = np.random.random()

        done = False
        epRewards = 0
        while not done:      
            # choice and validity check to avoid dead ends (loops)
            a, observation_, reward, done = choseandcheck(Q, a, observation, EPSILON, env, s)

            s_ = tuple(observation_)
            if observation_ not in states:
                states.append(observation_)
                for a in range(4):
                    #we changed the state to a to avoid a TypeError,
                    #because lists aren't hashable.
                    Q[tuple(s_),a] = 0
                    

            rand = np.random.random()
            if rand < (1-EPSILON):
                a_ = maxAction(Q,s_)
            else: 
                a_ = np.random.randint(0,4)
            epRewards += reward
            Q[s,a] = Q[s,a] + ALPHA*(reward + GAMMA*Q[s_,a_] - Q[s,a])
            
            # no more needed. that does not check for validity of move. we incorporated thi in choseandcheck
            # s,a = s_,a_

            #checks if the GOAL is reached. Sets the done to True to avoid KeyError (a higher state can be reached
            # but we don't want to reach it, because we don't have in in our state_space.)
            if get_max(observation_)==GOAL:
                game_won = True
            
            if get_max(observation_)>max_value:
                max_value = get_max(observation_)
                
            observation=observation_
        
        
        if game_won == True:
            won+= 1

        if EPSILON > 0.0002:
            EPSILON -= 2/(numGames)  
        else:
            EPSILON = 0.0002
        totalRewards.append(epRewards)
        max_total += get_max(observation)
        
        if (i+1)%(BATCH_SIZE) == 0:
            new = won/(BATCH_SIZE)
            num_won.append(new)
            max_values.append(max_value)
            average = max_total/BATCH_SIZE
            avg_values.append(average)
            won = 0
            max_value = 0
            max_total = 0
            print("Episode: " + str(i+1))    

In [ ]:
x = [j for j in range(1,101)]
lower_bound= (round-1)*100
upper_bound= round*100

In [ ]:
plotLearning(x,num_won[lower_bound:],'Batch','Won games in percentage', 'Win-Statistic')
# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(x,num_won[lower_bound:])

# use line_kws to set line label for legend
ax = sns.regplot(x=x, y=num_won[lower_bound:], color='b', 
 line_kws={'label':"y={0:.7f}x+{1:.7f}".format(slope,intercept)})

# plot legend
ax.legend()

plt.show()

plotLearning(x,max_values[lower_bound:],'Batch','Max_values', 'Max-Statistic')
# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(x,max_values[lower_bound:])

# use line_kws to set line label for legend
ax = sns.regplot(x=x, y=max_values[lower_bound:], color='b', 
 line_kws={'label':"y={0:.7f}x+{1:.7f}".format(slope,intercept)})

# plot legend
ax.legend()

plt.show()


plotLearning(x,avg_values[lower_bound:],'Batch','Avg_values', 'Average-Max-Statistic')
# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(x,avg_values[lower_bound:])

# use line_kws to set line label for legend
ax = sns.regplot(x=x, y=avg_values[lower_bound:], color='b', 
 line_kws={'label':"y={0:.7f}x+{1:.7f}".format(slope,intercept)})

# plot legend
ax.legend()

plt.show()


In [ ]:
#try to visualize overall progress
overall_episodes = round*numGames
print(overall_episodes)
boundary_overall =overall_episodes+1
x = [j for j in range(1, boundary_overall)]

plotLearning(x,num_won,'Batch','Won games in percentage', 'Win-Statistic')
# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(x,num_won)

# use line_kws to set line label for legend
ax = sns.regplot(x=x, y=num_won, color='b', 
 line_kws={'label':"y={0:.7f}x+{1:.7f}".format(slope,intercept)})

# plot legend
ax.legend()

plt.show()

plotLearning(x,max_values,'Batch','Max_values', 'Max-Statistic')
# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(x,max_values)

# use line_kws to set line label for legend
ax = sns.regplot(x=x, y=max_values, color='b', 
 line_kws={'label':"y={0:.7f}x+{1:.7f}".format(slope,intercept)})

# plot legend
ax.legend()

plt.show()


plotLearning(x,avg_values,'Batch','Avg_values', 'Average-Max-Statistic')
# get coeffs of linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(x,avg_values)

# use line_kws to set line label for legend
ax = sns.regplot(x=x, y=avg_values, color='b', 
 line_kws={'label':"y={0:.7f}x+{1:.7f}".format(slope,intercept)})

# plot legend
ax.legend()

plt.show()

